In [1]:
import numpy as np
import torch
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import time

In [2]:
data = pd.read_csv("853.csv")

data.head()

,caption,mean,precision,votes,not_funny,somewhat_funny,funny
0,Who knew the Swiss had a navy?,1.675661,0.018367,1816,946,513,357
1,Weapons down. It's the Swiss.,1.672884,0.020759,1394,721,408,265
2,"Wow, and that’s just the tip of the Jarlsberg!",1.643323,0.018380,1685,886,512,286
3,All we need now is to find a port,1.639823,0.012828,3837,2151,917,769
4,We must be directly over where Wisconsin used ...,1.638809,0.022989,1041,542,333,166


In [4]:
model =SentenceTransformer('msmarco-MiniLM-L-12-v3')
captions=data["caption"].tolist()
print(captions)

['Who knew the Swiss had a navy?', "Weapons down. It's the Swiss.", 'Wow, and that’s just the tip of the Jarlsberg!', 'All we need now is to find a port', 'We must be directly over where Wisconsin used to be.', "She's aged, but that's her alright.", "We're gonna need a bigger board.", 'We’re going to need a bigger board.', 'Sea Muenster.', 'That would pair well with a great white.', "Let's hope this is just the tip of the Jarlsberg.", 'No worries.  The Swiss are neutral', 'Careful men.  \nWe’re in un-charcuteried\nwaters.', 'It’ll be perfect with a great white.', 'I think we’re gonna need more crackers.', "Looks like we've entered Costco waters!", 'Careful boys, that might only be the tip of the Jarlsberg…', 'Careful—that could just be the tip of the Jarlsberg!', 'That can not be gouda.', "Careful boys, she's a sharp one.", 'Careful, men. It could be sharp!', 'Let it pass. We have Brie.', "Boys, we're gonna need a bigger board.", 'We must be getting close to Green Bay', 'Call me Bécham

In [5]:
captions_embds=model.encode(captions)

In [7]:
index = faiss.IndexFlatL2(captions_embds.shape[1])
index.add(captions_embds)
faiss.write_index(index, 'index_captions')
index = faiss.read_index('index_captions')

In [8]:
def search(query):
    
    t=time.time()
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [captions[_id] for _id in top_k[1].tolist()[0]]

In [17]:
results=search("wisconsin")
print(results)

totaltime: 0.05160689353942871
["Look! It's Wisconsin.", 'This usually only happens in Wisconsin!', 'We reached Wisconsin!', 'It is either Switzerland or Wisconsin', 'Wisconsin, in the year 2049.']
